In [1]:
from sample import sample_generator
from estimators import ClusteredSegmentation
from BarcodeScanner import cluster_barcode_scanner
import pandas as pd
import numpy as np

In [2]:
rng = sample_generator(3, 500, 2)
rng

sample_generator(p=3, sample_size=500, num_interactions=2, rng=Generator(PCG64) at 0x7FE422571820, beta_range=(3, 8), pi_range=(0.2, 0.8), error_scale=2.23606797749979)

In [3]:
num_cluster = np.unique(rng.barcode.reshape(-1)).shape[0]

In [4]:
cluster = ClusteredSegmentation(n_clusters = 3)#num_cluster)
cluster.fit(rng.barcode.reshape(-1,1), rng.y)
cluster.full_to_reduced_with_counts

,full,y,reduced,counts
0,0,-5.097076,0,59
1,1,-0.985655,1,51
2,2,-0.234910,1,151
3,3,11.690994,2,111
4,4,-8.826978,0,19
5,5,-5.083069,0,25
6,6,-4.140045,0,45
7,7,0.682918,1,39


In [5]:
cluster.tau_metric, cluster.tau_metric.tau_est

(tau(n=500, p=8, q=3, r_sqf=0.8756431219727703, r_sqr=0.8591840984272046, alpha=0.05),
 0.13235314207519572)

In [6]:
cluster.MSE

6.232168388595041

In [7]:
cluster.full_p

8

In [8]:
cluster.full_var.round(3)

array([[0.094, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.109, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.037, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.05 , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.293, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.222, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.124, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.143]])

### Snecdor's F test statistics

In [9]:
scanner = cluster_barcode_scanner(rng, cluster)

In [10]:
scanner._cdf_table = scanner.cdf_table.fillna(0)
scanner._cdf_table.cdf = scanner.cdf_table.cdf

In [11]:
scanner.cdf_table

,beta1*beta2,beta1*beta3,beta2*beta3,beta1*beta2*beta3,cdf
0,0,0,1,0,1.000000
1,0,0,1,1,1.000000
2,0,1,0,1,1.000000
3,0,1,1,0,1.000000
4,0,1,1,1,1.000000
5,1,0,0,1,1.000000
6,1,0,1,0,1.000000
7,1,0,1,1,1.000000
8,1,1,0,1,1.000000
9,1,1,1,0,1.000000


In [ ]:
scanner.cdf_table

In [13]:
scanner.cdf_ranking

,coefficients,score,ranking
0,beta2*beta3,0.630167,1.0
1,beta1*beta2*beta3,0.630167,2.0
2,beta1*beta3,0.512632,3.0
3,beta1*beta2,0.500966,4.0


In [ ]:
ranking_table = scanner.cdf_ranking

In [12]:
rng.interactions

{'X2*X3': 7.479393627725901, 'X1*X2*X3': -7.080352735799567}

In [ ]:
actual

In [ ]:
predicted

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, RocCurveDisplay

true_parameters = [x.replace('X','beta') for x in rng.interactions.keys()]
actual = ranking_table.coefficients.isin(true_parameters)
predicted = ranking_table.ranking.isin(list(range(1,11)))
RocCurveDisplay.from_predictions(actual, predicted)

In [ ]:
rng.interactions

In [190]:
import random
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, RocCurveDisplay
def simulation(sample_size, num_variables):
    num_all_interactions = 2**num_variables - num_variables -1
    k = random.randint(1, num_all_interactions - 1)
    rng = sample_generator(num_variables, sample_size, k)
    cluster = ClusteredSegmentation(n_clusters = 2**num_variables)
    cluster.fit(rng.barcode.reshape(-1,1), rng.y)
    scanner = cluster_barcode_scanner(rng, cluster, False) # <- need to change to False later
    scanner.set_beta_sum_range(1, 2)
#     ranking_table = scanner.get_distance_ranking(normalize = False)
    ranking_table = scanner.cdf_ranking
    true_parameters = [x.replace('X','beta') for x in rng.interactions.keys()]
    actual = ranking_table.coefficients.isin(true_parameters)
    predicted = ranking_table.ranking.isin(list(range(1, k+1)))
#     counts = ranking_table.loc[ranking_table.coefficients.isin(true_parameters),:].ranking.isin(list(range(1, k+1))).sum()
#     score = f1_score(actual, predicted)
#     score = roc_auc_score(actual, predicted)
    return (actual.to_numpy(), predicted.to_numpy())

In [ ]:
from joblib import Parallel, delayed

sample_size = 10_000

result = Parallel(n_jobs = -1, verbose = 10)(delayed(simulation)(sample_size = sample_size, num_variables = 5) for _ in range(100))


In [188]:
result[0]

<1x8 sparse array of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [189]:
result[0].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 1]])

### Misc

In [85]:
from scipy.sparse import csr_matrix, csr_array, vstack

In [86]:
from itertools import product
non_zero_part = product([0,1], repeat = 2**scanner.sample_generator.p - scanner.sample_generator.p - 1)
zeros = [0] + [0 for _ in range(scanner.sample_generator.p)]

In [87]:
from joblib import Parallel, delayed

In [88]:
def get_csr_array(x, zeros = zeros):
    return csr_array(zeros + list(x))

In [89]:
result = Parallel(n_jobs = -1, verbose = 10)(delayed(get_csr_array)(x) for x in non_zero_part if sum(x))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0054s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.0s finished


In [90]:
betas = vstack(result)

In [91]:
betas

<15x8 sparse matrix of type '<class 'numpy.int64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [92]:
betas.getrow(0)

<1x8 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [93]:
beta = csr_matrix(scanner.gen_beta_contrasts(betas.getrow(5).toarray()))

In [94]:
betas.getrow(0).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 1]])

In [95]:
beta.toarray()

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.70710678, -0.70710678,  0.        ]])

In [96]:
scanner.C

array([[ 0,  0,  1,  0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0, -1,  0],
       [ 1,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  1,  0, -1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0, -1]])

In [97]:
full_to_reduced = scanner.cluster_estimator.full_to_reduced_with_counts.copy()

In [98]:
full_to_reduced

,full,y,reduced,counts
0,0,-6.643216,1,172
1,1,-1.669044,2,55
2,2,-12.982594,0,83
3,3,-0.939035,2,17
4,4,-13.122382,0,89
5,5,-8.053988,1,40
6,6,-19.855971,0,40
7,7,-0.284431,2,4


In [104]:
y_hat = full_to_reduced.y
n_y = full_to_reduced.counts
y_tot = y_hat * n_y

In [107]:
C = scanner.C.dot(np.diag(y_tot))
ctc_inv = np.linalg.inv(C.dot(C.T))

In [108]:
H = (np.diag(np.ones(max(C.shape))) - C.T.dot(ctc_inv).dot(C))

In [110]:
L_inv = scanner.L_inv

In [112]:
beta.toarray()

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.70710678, -0.70710678,  0.        ]])

In [139]:
l_inv_b = L_inv.dot(beta.toarray().T)
denom = np.array(l_inv_b.T.dot(l_inv_b))
num   = np.array(l_inv_b.T.dot(H).dot(l_inv_b))


In [160]:
from numpy.linalg import inv, eig
eigens = eig(l_inv_b.dot(inv(l_inv_b.T.dot(l_inv_b))).dot(l_inv_b.T))

In [166]:
l_inv_b

array([[ 1.        ,  0.        ],
       [-1.        ,  0.        ],
       [ 0.        ,  0.70710678],
       [ 0.        , -0.70710678],
       [-1.        , -0.70710678],
       [ 1.        ,  0.70710678],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [167]:
eigens[0]

array([2.22044605e-16, 1.00000000e+00, 1.00000000e+00, 1.85037171e-17,
       4.62223187e-33, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [165]:
eigens[1].round(3)

array([[-0.816, -0.577, -0.04 ,  0.019, -0.016, -0.015,  0.   ,  0.   ],
       [-0.408,  0.577,  0.04 ,  0.716, -0.675, -0.612,  0.   ,  0.   ],
       [-0.204,  0.289,  0.519, -0.349,  0.565,  0.513,  0.   ,  0.   ],
       [ 0.204, -0.289, -0.519,  0.349, -0.094, -0.085,  0.   ,  0.   ],
       [-0.204,  0.289, -0.479, -0.349,  0.329,  0.596,  0.   ,  0.   ],
       [ 0.204, -0.289,  0.479,  0.349, -0.329, -0.002,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ]])

In [175]:
v = eigens[1][:,1:3]

In [176]:
l_inv_b.dot(inv(l_inv_b.T.dot(l_inv_b))).dot(l_inv_b.T).round(3)

array([[ 0.333, -0.333, -0.167,  0.167, -0.167,  0.167,  0.   ,  0.   ],
       [-0.333,  0.333,  0.167, -0.167,  0.167, -0.167,  0.   ,  0.   ],
       [-0.167,  0.167,  0.333, -0.333, -0.167,  0.167,  0.   ,  0.   ],
       [ 0.167, -0.167, -0.333,  0.333,  0.167, -0.167,  0.   ,  0.   ],
       [-0.167,  0.167, -0.167,  0.167,  0.333, -0.333,  0.   ,  0.   ],
       [ 0.167, -0.167,  0.167, -0.167, -0.333,  0.333,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [181]:
v.T[0]

array([-0.57735027,  0.57735027,  0.28867513, -0.28867513,  0.28867513,
       -0.28867513,  0.        ,  0.        ])

In [133]:
num/denom

0.23325860695200173

In [134]:
rng.interactions

{'X2*X3': 7.118762277547875, 'X1*X2*X3': 7.3032732545175065}

In [185]:
denom = np.array(v.T[1].dot(v.T[1].T))
num   = np.array(v.T[1].dot(H).dot(v.T[1].T))


In [186]:
num/denom

0.20531512169199037